In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pathlib
import kaleido
import plotly.express as px
import plotly.io as pio
from IPython.display import display, Markdown

pd.set_option('future.no_silent_downcasting', True)

In [3]:
# borne de recharge
data =  pd.read_csv(
    ".\data\consolidation-etalab-schema-irve-statique-v-2.3.1-20240705.csv",
    low_memory=False
    ,parse_dates=["date_mise_en_service"]
    ,dtype={"consolidated_code_postal": str
                       })\
  .sort_values(['id_pdc_itinerance', 'last_modified'])\

# Département
df_dep = pd.read_csv(
    ".\data\departements-france.csv"
    ,dtype={"code_departement": str
                        })



data.head()

,nom_amenageur,siren_amenageur,contact_amenageur,nom_operateur,contact_operateur,telephone_operateur,nom_enseigne,id_station_itinerance,id_station_local,nom_station,...,datagouv_dataset_id,datagouv_resource_id,datagouv_organization_or_owner,created_at,consolidated_longitude,consolidated_latitude,consolidated_code_postal,consolidated_commune,consolidated_is_lon_lat_correct,consolidated_is_code_insee_verified
0,Intermarché Yssingeaux,NaN,NaN,E-TOTEM,contact@e-totem.fr,NaN,Intermarché Yssingeaux,FRG10P43268A,FRG10P43268A,Intermarché Yssingeaux,...,6054b5221247124149d65773,f406e28c-1f83-4912-8884-6ae49c5ddc90,e-totem,2023-05-02T16:32:25.314000+00:00,4.116103,45.143782,43200,Yssingeaux,True,True
1,Hotel saint Alban,379629447.0,claudel.philippe@gmail.com,ELECTROMAPS,oscar@electromaps.com,34 931.574.967,Hotel saint alban,FR000011062174,02af2893-256f-4124-8d58-c728dc38f720,Hotel saint alban,...,627d1223d094836d7af7eb7f,19ebb40d-f992-457b-a93e-c0f7615e8a29,maneo,2022-05-12T15:56:52.355000+00:00,3.407609,43.419591,34120,Pézenas,False,True
2,Hôtel Restaurant Campanile Nogent-sur-Marne,349009423.0,manager.nogentsurmarne@campanile.fr,ELECTROMAPS,oscar@electromaps.com,34 931.574.967,Hôtel Restaurant Campanile Nogent-sur-Marne,FR000012292701,02af2893-256f-4124-8d58-c728dc38f720,Hôtel Restaurant Campanile Nogent-sur-Marne,...,627d1223d094836d7af7eb7f,19ebb40d-f992-457b-a93e-c0f7615e8a29,maneo,2022-05-12T15:56:52.355000+00:00,2.493570,48.832678,94130,Nogent-sur-Marne,True,True
3,Résidence Les Calanques,812328128.0,albarry@residence-lescalanques.fr,ELECTROMAPS,oscar@electromaps.com,34 931.574.967,Résidence les calanques,FR000012308585,02af2893-256f-4124-8d58-c728dc38f720,Résidence les calanques,...,627d1223d094836d7af7eb7f,19ebb40d-f992-457b-a93e-c0f7615e8a29,maneo,2022-05-12T15:56:52.355000+00:00,8.657888,41.908579,NaN,Ajaccio,True,True
4,1PACTE,803719277.0,m.levy@1pacte.com,ELECTROMAPS,oscar@electromaps.com,34 931.574.967,1PACTE,FR000012616553,02af2893-256f-4124-8d58-c728dc38f720,1PACTE,...,627d1223d094836d7af7eb7f,19ebb40d-f992-457b-a93e-c0f7615e8a29,maneo,2022-05-12T15:56:52.355000+00:00,5.476711,43.476584,13120,Gardanne,True,True


In [4]:
print(data.shape)
print(data.dtypes)

(80738, 51)
nom_amenageur                                  object
siren_amenageur                               float64
contact_amenageur                              object
nom_operateur                                  object
contact_operateur                              object
telephone_operateur                            object
nom_enseigne                                   object
id_station_itinerance                          object
id_station_local                               object
nom_station                                    object
implantation_station                           object
adresse_station                                object
code_insee_commune                             object
coordonneesXY                                  object
nbre_pdc                                        int64
id_pdc_itinerance                              object
id_pdc_local                                   object
puissance_nominale                            float64
prise_type_ef   

In [ ]:
# labéliser les colonnes pour les visuels
labels = {
    "nom_amenageur": "Nom amenageur"
    ,"nom_operateur": "Nom opérateur"
    ,"nom_enseigne": "Nom enseigne"
    ,"mis_en_service_cette_annee": "Mise en service en 2024"
    ,"date_mise_en_service": "Date de mise en service"
    ,"puissance_nominale_cat": "Puissance nominale"
    ,"prise_type_combo_ccs": "Possède une prise combo"
    ,"prise_type_2": "Possède une prise type 2"
}

In [ ]:
plt.figure(figsize=(40,20))
sns.heatmap(data.isna(), cbar=False)

In [ ]:
# Calculer les pourcentages de valeurs manquantes
missing_percentages = (data.isna().sum() / data.shape[0]).sort_values(ascending=False)

# Filtrer les colonnes avec un pourcentage de valeurs manquantes différent de zéro
filtered_missing_percentages = missing_percentages.loc[missing_percentages > 0]

# Afficher les pourcentages de valeurs manquantes filtrés
print(filtered_missing_percentages)

In [ ]:
# Suppression des colonnes qui ont plus que 50% des données manquantes
data = data.drop(columns=['id_pdc_local', 'observations', 'cable_t2_attache', 'tarification', 'num_pdl'])

# Suppression des colonnes qui ne peuvent pas etre remplises () et surtout ils ont une pourcentage importante manquante 
data =  data.drop(columns=['id_station_local'])

# le pourcentage des valeurs manquantes dans les colonnes suivantes est faible 
data.dropna(subset=['contact_amenageur', 'telephone_operateur'], inplace=True)


In [ ]:
# Verifier les pourcentages après les drop
# Shows the precentge of missing values in each column
(data.isna().sum()/data.shape[0]).sort_values(ascending=True)

In [ ]:
# Convertir les valeurs en chaînes de caractères, obtenir les valeurs uniques et les trier par ordre ascendant
unique_data_paiment_cb = sorted(data['paiement_cb'].astype(str).unique())
unique_data_paiment_autre = sorted(data['paiement_autre'].astype(str).unique())
unique_data_gratuit = sorted(data['gratuit'].astype(str).unique())
unique_data_raccordemant = sorted(data['raccordement'].astype(str).unique())

# Afficher les valeurs uniques
print(unique_data_paiment_cb)
print('\n')
print(unique_data_paiment_autre)
print('\n')
print(unique_data_gratuit)
print('\n')
print(unique_data_raccordemant)

In [ ]:
# fill it with random values from the column itself paiment_cb, paiment_autre, raccordemant 

In [ ]:
df_filter = data["id_pdc_itinerance"]=="Non concerné"

data = pd.concat([
  data[~df_filter].drop_duplicates('id_pdc_itinerance', keep='last'),
  data[df_filter]
])

In [ ]:
# Calcul approximatif du nombre de PDC par année
# Extraire l'année à partir de la colonne de dates
data['annee'] = data['date_mise_en_service'].dt.year

# Compter le nombre de points de recharge par année
count_by_year = data['annee'].value_counts().sort_index()

# Afficher le résultat
print("Nombre de points de recharge par année :")
print(count_by_year)

In [ ]:
# Date de mise en service
# Remplacer les valeurs de mise en service avant 2010 par None
data.loc[data["date_mise_en_service"]<pd.to_datetime("2010-01-01"),"date_mise_en_service"]=None
# Vérifier si la mise en service a eu lieu en 2024 en ajoutant une colonne 'mis_en_service_cette_annee'
data["mis_en_service_cette_annee"] = np.where(data["date_mise_en_service"].dt.year==pd.to_datetime("2024-01-01").year
         , "Oui"
         , "Non")

In [ ]:
# Power
# Normaliser les valeurs de la puissance et les catégoriser
data["puissance_nominale_cat"] = pd.cut(\
  data["puissance_nominale"].apply(lambda x: x/1000 if x >1000 else x)\
,[0,1.8,3.5,7.5,26,52,151,500]
,labels=["1.7","3.4","7.5","22","50","150",">150"]
, include_lowest=False)

In [ ]:
# les types de prise 
data["prise_type_combo_ccs"] = (data["prise_type_combo_ccs"].str.lower().map
                      ({"0":"Non","false":"Non","1":"Oui","true":"Oui"}))
data["prise_type_2"] = (data["prise_type_2"].str.lower().map
                      ({"0":"Non","false":"Non","1":"Oui","true":"Oui"}))
# les types de paiment 
data["paiement_autre"] = (data["paiement_autre"].str.lower().map
                      ({"0":"FALSE","false":"FALSE","False":"FALSE","1":"TRUE","true":"TRUE","True":"TRUE"}))
data["paiement_cb"] = (data["paiement_cb"].str.lower().map
                      ({"0":"FALSE","false":"FALSE","False":"FALSE","1":"TRUE","true":"TRUE","True":"TRUE"}))

data["gratuit"] = (data["gratuit"].str.lower().map
                      ({"0":"FALSE","false":"FALSE","False":"FALSE","1":"TRUE","true":"TRUE","True":"TRUE"}))



In [ ]:
# Ajout d'une colonne 'code_departement' à partir de 'consolidated_code_postal'
data["code_departement"] = data["consolidated_code_postal"].str[:2]

# Join des deux tables sur la colonne 'code_departement'
data = pd.merge(data,df_dep, how='left', on='code_departement')

In [ ]:
data.head(100)

In [ ]:
data.columns

In [ ]:
# Shows the precentge of missing values in each column
(data.isna().sum()/data.shape[0]).sort_values(ascending=True)

In [ ]:
# Convertir les valeurs en chaînes de caractères, obtenir les valeurs uniques et les trier par ordre ascendant
unique_data_paiment_cb = sorted(data['paiement_cb'].astype(str).unique())
unique_data_paiment_autre = sorted(data['paiement_autre'].astype(str).unique())
unique_data_gratuit = sorted(data['gratuit'].astype(str).unique())
unique_data_raccordemant = sorted(data['raccordement'].astype(str).unique())

# Afficher les valeurs uniques
print(unique_data_paiment_cb)
print('\n')
print(unique_data_paiment_autre)
print('\n')
print(unique_data_gratuit)
print('\n')
print(unique_data_raccordemant)

In [ ]:
# le but de la fonction c'est de remplacer les valeurs nan par des randoms values pour les remplir 
def replace_nan_with_random(df, column):
    # Sélectionner les valeurs non manquantes
    non_nan_values = df[column].dropna().unique()
    # Générer des valeurs aléatoires pour remplacer les NaN
    random_values = np.random.choice(non_nan_values, size=df[column].isna().sum(), replace=True)
    # Remplacer les NaN par les valeurs aléatoires générées
    df.loc[df[column].isna(), column] = random_values
    

In [ ]:
# Remplacer les valeurs NaN par des valeurs aléatoires pour chaque colonne spécifiée
for column in ['paiement_cb', 'paiement_autre', 'raccordement']:
    replace_nan_with_random(data, column)

In [ ]:
# Shows the precentge of missing values in each column
(data.isna().sum()/data.shape[0]).sort_values(ascending=True)

In [ ]:
print(data[['date_mise_en_service', 'mis_en_service_cette_annee']].head(100))


In [ ]:
print(data.dtypes)

In [ ]:
numeric_columns = data.select_dtypes(include=[np.number])

# Calcul de la matrice de corrélation
correlation_matrix = numeric_columns.corr()

# Affichage de la matrice de corrélation avec un heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', square=True)
plt.title('Matrice de corrélation')
plt.show()

In [ ]:
# Sélection des colonnes pertinentes
x = data['nbre_pdc']
y = data['puissance_nominale']

# Création du scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(x, y, alpha=0.5)
plt.title('Corrélation entre nbre_pdc et puissance nominale')
plt.xlabel('Nombre de points de charge')
plt.ylabel('Puissance nominale')
plt.grid(True)
plt.show()

In [5]:

file_path = '.\\data\\voitures-par-commune-par-energie.csv'

dtype_dict = {
    'codgeo': str,          
    'epci': str,            
}

try:
    df_ve = pd.read_csv(file_path, delimiter=';', dtype=dtype_dict, on_bad_lines='warn', quoting=3)
except Exception as e:
    print(f"Erreur lors du chargement du fichier : {e}")

# Afficher les premières lignes pour vérifier le chargement
print(df_ve.head())

# Vérifier les colonnes disponibles
print(df_ve.columns)


  codgeo  libgeo       epci                      libepci date_arrete  \
0  15022  BONNAC  200066637  CC Hautes Terres Communauté  31/12/2021   
1  15022  BONNAC  200066637  CC Hautes Terres Communauté  31/03/2022   
2  15022  BONNAC  200066637  CC Hautes Terres Communauté  30/09/2022   
3  15022  BONNAC  200066637  CC Hautes Terres Communauté  31/12/2022   
4  15022  BONNAC  200066637  CC Hautes Terres Communauté  31/03/2024   

   nb_vp_rechargeables_el  nb_vp_rechargeables_gaz  nb_vp  
0                       0                        0    221  
1                       0                        0    222  
2                       0                        0    226  
3                       0                        0    223  
4                       0                        0    236  
Index(['codgeo', 'libgeo', 'epci', 'libepci', 'date_arrete',
       'nb_vp_rechargeables_el', 'nb_vp_rechargeables_gaz', 'nb_vp'],
      dtype='object')


In [6]:
df_ve= df_ve.drop(['nb_vp_rechargeables_gaz', 'libepci', 'epci'], axis=1)


In [7]:
print(df_ve.shape)
print(df_ve.dtypes)

(492532, 5)
codgeo                    object
libgeo                    object
date_arrete               object
nb_vp_rechargeables_el     int64
nb_vp                      int64
dtype: object
